In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Acquire the data from a .csv file that was saved locally
#For more information on acquiring the data for replication, please refer to the README in this repo
df = pd.read_csv('open-broadway-data 2021-04-08.csv')

In [ ]:
#Preview the dataframe
df.head(1)

## Data Preparation

In [ ]:
#What does the dataframe contain?
df.info()

In [ ]:
#Stats for the dataframe
df.describe()

In [ ]:
#Confirm show id is unique so that it can become the index
df.show_id.nunique()

In [ ]:
#df = df.set_index('show_id', 'Show Title')


In [ ]:
df.head()

**Takeaways and next steps:**
- Check if show_id is unique and can be reset to index
- A lot of nulls in previews date ==> Drop this column?
- Some missing opening dates ==> Were these shows that never opened?
- A LOT of missing closing dates...this may actually affect the target if it can't be imputed
- Small number of missing theatre names...can I impute from other columns?
- Missing values in intermissions...does that mean there wasn't an intermission or is that data missing?
- Missing vales in  N performances...but more data points than closing date...may be helpful as a secondary target if I lose too much data due to missing values in closing date
- Drop run time
- Other titles missing... use domain knowledge to verify I can just drop this column
- Drop Official website under the assumption that it doesn't impact the target
- theatre_id...probably captured in theatre name
- Address info...keep one feature and drop the others? Location may matter, but maybe that is captured or can be inferred from theatre name? / Engineer a feature for visibility of theatre /promity to Broadway? May need to be done in future iteration but can see how this could be a driver of length of run.
- Some theatres closed...might be an interesting feature that supports drivers of length of run?
- Theatre capacity is definitely a nice feature, missing nulls...maybe impute with mean, median or mode?

In [ ]:
#Does opening data have any missing values that could be filled by previews date?

#Check to see what values have have missing opening and previes dates...
df[df['Opening Date'].isna() & df['Previews Date'].isna()]


In [ ]:
#Create a separate datadframe to explore this...
df_no_close = df[df['Closing Date'].isna()]

In [ ]:
df_no_close.info()

In [ ]:
df_no_close.describe()

In [ ]:
df_no_close[df_no_close["Closing Date"].isna() & (df_no_close['N Performances'] > 0)]

**Questions**<br>
    - There are less missing values in number of performances than there are in closing date...<br>
        - Can I infer the number of weeks a show ran by number of performances? Assuming the standard broadway schedule?

In [ ]:
df_no_close[df_no_close['Show Not Opened'] == False]

In [ ]:
df.drop(columns={'Previews Date', 'Intermissions', 'N Performances', 'Run Time', 'Other Titles', 'Official Website', 
                 'Theatre Postal Code', 'Theatre Year Closed','Theatre Year Demolished'}, inplace=True)

In [ ]:
df.info()

In [ ]:
#df['Theatre Year Closed'].fillna('False', inplace=True)


In [ ]:
df.info()

In [ ]:
#df['Theatre Year Demolished'].fillna('False', inplace=True)

In [ ]:
df.info()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.info()

## Data Exploration

In [ ]:
df.Year.hist()

In [ ]:
df['Production Type'].hist()

In [ ]:
df['Show Type (Simple)'].hist()


In [ ]:
df['N People'].hist()

In [ ]:
df['N Performers'].hist()

In [ ]:
df['Theatre Capacity'].hist()

In [ ]:
df['Closing Date']

In [ ]:
df['length_of_run'] = pd.to_datetime(df['Closing Date'], format='%Y-%m-%d') - pd.to_datetime(['Opening Date'], format='%Y-%m-%d')

In [ ]:
df['Closing Date'] = pd.to_datetime(df['Closing Date'], format='%Y-%m-%d')

In [ ]:
df.info()

In [ ]:
pd.to_datetime(['Opening Date'], errors='coerce')

In [ ]:
df['Opening Date'].sample(20)

In [ ]:
df['Opening Date'] = pd.to_datetime(df['Opening Date'], format='%Y-%m-%d')

In [ ]:
df.info()

In [ ]:
df['length_of_run'] = df['Closing Date'] - df['Opening Date']

In [ ]:
df.info()

In [ ]:
df.sample(20)

In [ ]:
df.length_of_run.astype('int64')

In [ ]:
df['length_of_run'] = df.length_of_run.astype('str')

In [ ]:
df['length_of_run'] = df['length_of_run'].str.replace(r'\D', '').astype(int)

In [ ]:
df.length_of_run.hist()

In [ ]:
df.length_of_run

In [ ]:
df.length_of_run.median()

In [ ]:
df.describe()

In [ ]:
#What is the show with the max run?
df[df.length_of_run == 7293]

**Domain Knowledge Question:**
Never heard of the show called The Peri...And dates are inverted...Is this accurate?

In [ ]:
#How many unique show types are there?
len(df['Show Type'].unique())

In [ ]:
df['Show Type'].unique()

**Takeaways on show types**
- Beneifts are usually one off events...confirm this is true and if  so, drop those observations?

In [ ]:
index_names = df[df['Show Type'] == 'Benefit'].index

In [ ]:
df.drop(index_names, inplace=True)

In [ ]:
df['Show Type'].unique()

In [ ]:
df.info()

In [ ]:
index_locality = df[df['Theatre Address Locality'] != 'New York'].index

In [ ]:
df.drop(index_locality, inplace = True)

In [ ]:
df.shape

In [ ]:
#Create Dummie for if the show is a revival or not
df['is_revival'] = df['Revival'].apply(lambda x: 1 if x == True else 0 )
df = df.drop(columns={'Revival'}, inplace=True)

In [ ]:
df.info()

In [ ]:
df.Revival.unique()

### Tasks to do before Exploration:

In [ ]:
#Encode Show_Type (Simple)